In [6]:
import pandas as pd
import numpy as np
import datetime
import re
from fuzzywuzzy import fuzz



In [8]:
df = pd.read_excel("Pub provided awards 8-14-2023 to 8-28-2023.xlsx")

In [9]:
df = df.rename(columns={'newvalue':'Publisher-Provided Awards'})

In [10]:
df_NYT_1 = pd.read_excel("NYT ids 8-21-2023.xlsx")
df_NYT_2 = pd.read_excel("NYT ids 8-28-2023.xlsx")

In [11]:
award = df.loc[1, 'Publisher-Provided Awards'].split("|")
award

['2015 Audie Award Finalist Short-listed']

In [12]:
df['Removed Non-Winners Count'] = ""
df['Winner List'] = ""
df_x = df[['content_id','Publisher-Provided Awards','Removed Non-Winners Count','Winner List']]

In [13]:
award_list = ("Oprah's Book Club","Oprah's Book Club","YALSA Amazing Audiobooks for Young Adults","YALSA Best of the Best Winner","Hugo Award Winner","Caldecott Medal","Grammy Award","Man Booker Prize Winner","ALSC Notable Children's Recording","Alex Award Winner","Newbery Honor","Pulitzer Prize Winner","Edgar Award Winner","National Book Critics Circle Award Winner","Nebula Award Winner","PW Listen-Up Award Winner","Audie Award Winner","Nobel Prize Winner","Library Journal Best Audiobooks of Year","AudioFile Earphone Award Winner","Carnegie Medal in Literature","Newbery Medal Winner","National Book Award Winner","RUSA Notable Book","YALSA Amazing Audiobook","RITA Award","ITW Thriller Award","Odyssey Award","Coretta Scott King Award","Pura Belpre Award","Printz Award","William C. Morris Award","Sibert","Stonewall","American Indian Youth Literature","Asian / Pacific American Award","Sidney Taylor Book Award","Reese's Book Club","Good Morning America Book Club","YALSA Award Winner","Audie Audiobook of the Year Winner")
award_min = ("Oprah's Book Club","Oprah Book Club","Amazing Audiobooks for Young Adults","YALSA Best of the Best","Hugo","Caldecott Medal","Grammy","Man Booker","ALSC Notable Children's Recording","Alex","Newbery Honor","Pulitzer","Edgar","National Book Critics Circle","Nebula","PW Listen-Up","Audie","Nobel","Library Journal Best Audiobooks of Year","AudioFile Earphone Award","Carnegie Medal in Literature","Newbery Medal","National Book","RUSA Notable Book","YALSA Amazing Audiobook","RITA","ITW Thriller","Odyssey","Coretta Scott King","Pura Belpre","Printz","William C. Morris","Sibert","Stonewall","American Indian Youth Literature","Asian / Pacific American","Sidney Taylor Book","Reese's Book Club","Good Morning America Book Club","YALSA","Audie Audiobook of the Year")

In [14]:
index = award_list.index("National Book Award Winner")
index

22

In [15]:
t =21
print(award_list[t])
print(award_min[t])

Newbery Medal Winner
Newbery Medal


In [16]:
def normalize(award):
    for n in award_min:        
        x=""
        if fuzz.partial_ratio(n.lower(), award.lower()) > 90:
            x = award
            index =0
            if n in award_min:
                index = award_min.index(n)
                z = award_list[index]  
            
                return z  
      

In [17]:

for index, row in df_x.iterrows():
    winner_list =[]
    winner_final = []
    winner_final_s = []
    count =0
    award = df_x.loc[index, 'Publisher-Provided Awards'].split("|")

    for a in range(len(award)):
        if "Nominated" in award[a]:
            count+= 1
        elif "Short-listed" in award[a]:
            count+= 1
        elif "Long-listed" in award[a]:
            count+= 1
        elif "Commended" in award[a]:
            count+= 1
        elif "Finalist" in award[a]:
            count+= 1
        elif "Runner-up" in award[a]:
            count+= 1
        elif "winning author" in award[a]:
            count+= 1
        else:
            s = re.sub("\d\d\d\d", "", award[a]).strip()#.capitalize()
            normalized = normalize(s)
            winner_list.append(normalized)
            winner_final = [i for i in winner_list if i] # removes empty indices
            winner_final = (list(set(winner_final)))

    winners = str(winner_final).strip()
    df_x.loc[index, 'Removed Non-Winners Count'] = count
    df_x.loc[index, 'Winner List'] = winners  


In [18]:
today = pd.to_datetime('today')
end_day = str(today)[:10]
start_day = str(today - pd.to_timedelta(14, unit='d'))[:10]

In [19]:
df_x.to_excel (r'Awards '+ start_day + ' to ' + end_day + '.xlsx', index = False, header=True)

## add NYT Awards

In [22]:
NYT_combo_a = pd.concat([df_NYT_1, df_NYT_2], ignore_index=True)
NYT_combo_b = NYT_combo_a[['content_id']]
NYT_Award_list = NYT_combo_b.drop_duplicates()

In [23]:
NYT_Award_list.to_csv (r'NYT_award list ' + end_day + '.csv', index = False, header=True,encoding='utf-8-sig')

In [24]:
NYT_Award_list = NYT_combo_b.drop_duplicates()
NYT_Award_list.head()

,content_id
0,1202757
1,1225555
2,417239
3,329823
4,1175109


In [25]:
pd.Timestamp.now()

Timestamp('2023-08-28 10:57:49.627853')